# Skills gaps analysis

# 0. Import dependencies and inputs

In [2]:
%run ../notebook_preamble_Transitions.ipy
import os

data = load_data.Data()
sims = load_data.Similarities()

In [3]:
file_name = 'filter_matrices_Report_occupations.pickle'
export_path = f'{data_folder}interim/transitions/{file_name}'

if os.path.exists(export_path):
    filter_matrices = pickle.load(open(export_path,'rb'))
    print(f'Imported filtering matrices from {export_path}')


Imported filtering matrices from /Users/karliskanders/Documents/mapping-career-causeways/codebase/data/interim/transitions/filter_matrices_Report_occupations.pickle


In [ ]:
# import pandas as pd
# import numpy as np
# import pickle
# from tqdm.notebook import tqdm 
# from time import time

# import career_causeways.compare_nodes_utils as compare_nodes_utils
# find_closest = compare_nodes_utils.find_closest
# import career_causeways.load_data as load_data

# from itertools import combinations

# import umap
# import matplotlib.colors as colors
# from matplotlib.colors import ListedColormap
# import seaborn as sns
# from matplotlib import pyplot as plt
# import plotly.express as px
# from scipy.spatial.distance import pdist, squareform

In [ ]:
# # Import data
# data_folder = '../../data/'
# outputs_folder = '../../reports/outputs/'

# filter_matrices = pickle.load(open(data_folder + 'processed/filter_matrices_Top_occupations.pickle','rb'))
# for key in filter_matrices.keys():
#     print(key)

In [ ]:
# # Import occupations and skills tables
# occupations = pd.read_csv(data_folder + 'processed/item_occupations.csv')
# skills = pd.read_csv(data_folder + 'processed/clusters/Official_ESCO_hierarchy_all_skills.csv')
# concepts = pd.read_csv(data_folder + 'processed/clusters/Official_ESCO_hierarchy.csv')
# occupation_to_skills = pd.read_csv(data_folder + 'processed/link_occupation_to_skills.csv')

# # Import occupation data

# # Hierarchy
# occ_hierarchy = pd.read_csv(data_folder + 'processed/clusters/ESCO_occupation_hierarchy/occupations_ESCO_hierarchy.csv')
# isco_titles = pd.read_csv(data_folder + 'processed/clusters/ISCO_level_titles.csv')

# occ_risk = pd.read_csv(data_folder + 'processed/links/ESCO_occupations_Automation_risk.csv')
# occ_risk.loc[occ_risk.risk_cat_label.isnull(), 'risk_cat_label'] = 'unknown'

# occ_clust = pd.read_csv(data_folder + 'processed/clusters/ESCO_occupation_clusters_v1_1_curated.csv')
# clusters_level_1 = pd.read_csv(data_folder + 'processed/clusters/ESCO_occupation_clusters_v1_1_LEVEL1.csv')
# clusters_level_2 = pd.read_csv(data_folder + 'processed/clusters/ESCO_occupation_clusters_v1_1_LEVEL2.csv')

# occ_exp = pd.read_csv(data_folder + 'processed/links/ESCO_occupations_Job_zone_Education.csv')
# occ = occ_risk.merge(occ_exp[['concept_uri','job_zone','education_level']], on='concept_uri', how='left')
# occ = occ.merge(occ_hierarchy[['id', 'isco_level_1', 'isco_level_2','isco_level_3', 'is_top_level']], on='id', how='left')


# # Skills description embeddings
# embeddings = np.load(data_folder + 'interim/embeddings/embeddings_skills_description_SBERT.npy')

# # Import dataframe with skills IDs for each occupation
# node_to_essential_items = pickle.load(open(data_folder + 'processed/occupation_to_essential_skills.pickle','rb'))
# node_to_all_items = pickle.load(open(data_folder + 'processed/occupation_to_all_skills.pickle','rb'))

# # Import the similarity matrix
# W_combined = np.load(data_folder + 'processed/sim_matrices/occupationSimilarity_Combined.npy')
# W_essential = np.load(data_folder + 'processed/sim_matrices/occupationSimilarity_EssentialSkillsDescription_asymmetric.npy')
# # Comparison of all (at origin) vs. essential skills (at destination)
# W_all_to_essential = np.load(data_folder + 'processed/sim_matrices/occupationSimilarity_AllToEssentialSkillsDescription_asymmetric.npy')
# # Similarities based on ESCO hierarchy (work activities)
# W_esco_level_2 = np.load(data_folder + 'processed/sim_matrices/occupationSimilarity_ESCO_clusters_Level_2.npy')

# # Similarities based on ONET's work context
# # (NB: There are occupations that didn't have a defined work context)
# W_work_context = np.load(data_folder + 'processed/sim_matrices/occupationSimilarity_ONET_Work_Context.npy')

# # Import skills clusters
# skills_clust = pd.read_csv(data_folder + 'processed/clusters/ESCO_skills_clusters/ESCO_Essential_clusters_May2020_coreness.csv')

# # Import skills to occupaitons adjacencies
# A_ess = np.load(data_folder + 'processed/sim_matrices/A_ess.npy')
# A_opt = np.load(data_folder + 'processed/sim_matrices/A_opt.npy')


In [ ]:
# # Number of transitions
# occ_n_trans = pd.read_csv(outputs_folder + 'transitions_outputs/occupations_number_of_transitions.csv')


In [ ]:
# def occ_skills(j_id, importance=['Essential']):
#     """
#     Get the skills list of a given occupation
#     """
#     df = occupation_to_skills[(occupation_to_skills.occupation_id==j_id) & 
#                               (occupation_to_skills.importance.isin(importance))]
#     return skills.loc[df.skill_id.to_list()]

In [ ]:
# top_occ = occ[occ.is_top_level==True].id.to_list()


In [ ]:
# VIABLE_THRESH_0 = 0.30
# VIABLE_THRESH_1 = 0.40
# VIABLE_THRESH_2 = 0.60
# THRESH = [VIABLE_THRESH_0, VIABLE_THRESH_1, VIABLE_THRESH_2]

In [ ]:
# # Define origin and destination occupations to analyse
# id_to_analyse = occ[
#     (occ.risk_cat_label=='Very high risk') &
#     (occ.is_top_level==True)].id.values

# # Select only top-level occupations for transition analysis (which have risk estimates)
# top_occ = occ[(occ.is_top_level==True) & (occ.risk.isnull()==False)].id.to_list()
# assert(len(top_occ)==1627)
      
# # Define dataframe with the origin occupations to analyse
# occ_to_analyse = occ.loc[id_to_analyse].copy()
# assert(len(occ_to_analyse)==217)

In [ ]:
# # Import the transitions
# transitions = pd.read_csv(outputs_folder + 'transitions_all_occupations.csv')
# trans = transitions[transitions.is_safe_destination]

# trans = trans[trans.origin_id.isin(id_to_analyse) & (trans.destination_id.isin(top_occ))].copy()
# print(len(trans))

# trans.head(1)

In [21]:
# import importlib
# importlib.reload(trans_utils)

## 0.1 Generate the transitions to analyse

In [4]:
high_risk_ids = data.occ_report[data.occ_report.risk_category=='High risk'].id.to_list()
transition_df = trans_utils.get_transitions(origin_ids=high_risk_ids, destination_ids='report')
trans_to_analyse = transition_df[transition_df.is_strictly_safe_desirable]

Finding all transitions... Done!
This took 3.12 seconds.


# 1. Obtain matching scores for all skills involved in the transitions

In [5]:
# # Import dataframe with the skills IDs for each occupation
# node_to_essential_items = pickle.load(open(data_folder + 'processed/occupation_to_essential_skills.pickle','rb'))
# node_to_all_items = pickle.load(open(data_folder + 'processed/occupation_to_all_skills.pickle','rb'))


In [6]:
print(f"Analysis of {len(trans_to_analyse)} transitions")

Analysis of 3619 transitions


In [7]:
import importlib
importlib.reload(trans_utils)
importlib.reload(load_data)


<module 'mapping_career_causeways.load_data_utils' from '/Users/karliskanders/Documents/mapping-career-causeways/mapping_career_causeways/load_data_utils.py'>

In [16]:
df = trans_utils.show_skills_overlap('hotel porter','shop manager')

from hotel porter to shop manager
--------
18/27 destination skills matched
NLP-adjusted overlap = 0.05 (total combined similarity: 0.19)


In [20]:
df[df.origin_skill_id.apply(lambda x: type(x)!=str)]

,origin_skill_id,origin_skill,destination_skill_id,destination_skill,similarity,score
0,6288,maintain customer service,12494,monitor customer service,0.908888,0.875
1,9777,implement sales strategies,11989,supervise merchandise displays,0.854552,0.316
2,1020,implement marketing strategies,1737,order supplies,0.825074,0.096
3,4351,handle chemical cleaning agents,11611,apply health and safety standards,0.809382,0.046
4,2861,provide tourism related information,1809,maintain relationship with customers,0.791304,0.019
5,2187,handle customer complaints,7321,measure customer feedback,0.765628,0.005
6,9640,assist clients with special needs,9007,adhere to organisational guidelines,0.757962,0.004
7,11855,handle delivered packages,5560,oversee promotional sales prices,0.734061,0.001
8,6969,run errands on behalf of customers,3659,sales activities,0.717332,0.000
9,3467,provide door security,12591,manage theft prevention,0.710784,0.000


In [30]:
data.occupation_skills(1, skill_importance='Essential')

,occupation_id,skill_id,importance,preferred_label,description
0,1,6763,Essential,types of metal,"Qualities, specifications, applications and re..."
1,1,8417,Essential,supply machine,Ensure the machine is fed the necessary and ad...
2,1,2687,Essential,remove inadequate workpieces,Evaluate which deficient processed workpieces ...
3,1,7508,Essential,quality standards,"The national and international requirements, s..."
4,1,7162,Essential,dies,The qualities and applications of various kind...
5,1,5640,Essential,ensure equipment availability,Ensure that the necessary equipment is provide...
6,1,9619,Essential,cold drawing processes,The various kinds of metalworking drawing proc...
7,1,2756,Essential,troubleshoot,"Identify operating problems, decide what to do..."
8,1,2156,Essential,monitor gauge,Oversee the data presented by a gauge concerni...
9,1,3154,Essential,quality and cycle time optimisation,The most optimal rotation or cycle time and ov...


In [ ]:
def get_matching_scores(trans_to_analyse):
    
    # All transitions
    skills_id_ALL = []
    skills_origin_id_ALL = []
    # All matching scores
    skills_id_sim_ALL = []
    skills_origin_id_sim_ALL = []
    # All semantic similarity values (not used in the final analysis)
    skills_id_raw_sim_ALL = []
    skills_origin_id_raw_sim_ALL = []

    matched_skills_origin = []
    matched_skills_destination = []
    matched_skills_similarity = []
    matched_skills_similarity_raw = []

    t = time()
    for j, row in tqdm(trans_to_analyse.iterrows(), total=len(trans_to_analyse)):

        # Get job IDs
        job_i = row.origin_id
        job_j = row.destination_id

        # Create the input dataframe in the required format
        node_to_items_ = pd.concat([node_to_all_items.loc[[job_i]],
                                    node_to_essential_items.loc[[job_j]]])

        # Compare jobs
        df, score = compare_nodes_utils.two_node_comparison(
            node_to_items_,
            job_i, job_j,
            skills[['id','preferred_label']],
            embeddings,
            metric='cosine',
            matching_method='one_to_one',
            symmetric=False)

        ###### DESTINATION SKILLS ######
        # Include unmatched skills as well
        destination_skills = pd.DataFrame(data={'destination_id': occ_skills(job_j).id.to_list()})
        destination_skills = destination_skills.merge(df[['id_y', 'similarity', 'similarity_raw']], left_on='destination_id', right_on='id_y', how='left')
        destination_skills.loc[destination_skills.similarity.isnull(), 'similarity'] = 0
        destination_skills.loc[destination_skills.similarity.isnull(), 'similarity_raw'] = 0

        # Extract the skill IDs and similarity values
        skills_id = destination_skills.destination_id.to_list()
        skills_id_sim = destination_skills.similarity.to_list()
        skills_id_sim_raw = destination_skills.similarity_raw.to_list()  

        # Save the skill IDs and similarity values
        skills_id_ALL.append(skills_id)
        skills_id_sim_ALL.append(skills_id_sim)
        skills_id_raw_sim_ALL.append(skills_id_sim_raw)

        ###### ORIGIN SKILLS ######
        # Include unmatched skills as well
        origin_skills = pd.DataFrame(data={'origin_id': occ_skills(job_i, ['Essential', 'Optional']).id.to_list()})
        origin_skills = origin_skills.merge(df[['id_x', 'similarity', 'similarity_raw']], left_on='origin_id', right_on='id_x', how='left')
        origin_skills.loc[origin_skills.similarity.isnull(), 'similarity'] = 0
        origin_skills.loc[origin_skills.similarity.isnull(), 'similarity_raw'] = 0

        # Extract the skill IDs and similarity values
        skills_origin_id = origin_skills.origin_id.to_list()
        skills_origin_id_sim = origin_skills.similarity.to_list()
        skills_origin_id_sim_raw = origin_skills.similarity_raw.to_list()  

        # Save the skill IDs and similarity values
        skills_origin_id_ALL.append(skills_origin_id)
        skills_origin_id_sim_ALL.append(skills_origin_id_sim)
        skills_origin_id_raw_sim_ALL.append(skills_origin_id_sim_raw)    

        ###### MATCHED SKILLS ######
        # Include unmatched skills as well
        matched_skills_origin.append(df.id_x.to_list())
        matched_skills_destination.append(df.id_y.to_list())
        matched_skills_similarity.append(df.similarity.to_list())
        matched_skills_similarity_raw.append(df.similarity_raw.to_list())


    t_elapsed = time() - t 
    print(f'Time elapsed: {t_elapsed :.2f} sec ({t_elapsed/len(trans_to_analyse): .3f} per transition)')


In [31]:
# # All transitions
# skills_id_ALL = []
# skills_origin_id_ALL = []
# # All matching scores
# skills_id_sim_ALL = []
# skills_origin_id_sim_ALL = []
# # All semantic similarity values (not used in the final analysis)
# skills_id_raw_sim_ALL = []
# skills_origin_id_raw_sim_ALL = []

# matched_skills_origin = []
# matched_skills_destination = []
# matched_skills_similarity = []
# matched_skills_similarity_raw = []


# t = time()
# for j, row in tqdm(trans_to_analyse.iterrows(), total=len(trans_to_analyse)):
    
#     # Get job IDs
#     job_i = row.origin_id
#     job_j = row.destination_id
    
#     # Create the input dataframe in the required format
#     node_to_items_ = pd.concat([node_to_all_items.loc[[job_i]],
#                                 node_to_essential_items.loc[[job_j]]])
    
#     # Compare jobs
#     df, score = compare_nodes_utils.two_node_comparison(
#         node_to_items_,
#         job_i, job_j,
#         skills[['id','preferred_label']],
#         embeddings,
#         metric='cosine',
#         matching_method='one_to_one',
#         symmetric=False)

#     ###### DESTINATION SKILLS ######
#     # Include unmatched skills as well
#     destination_skills = pd.DataFrame(data={'destination_id': occ_skills(job_j).id.to_list()})
#     destination_skills = destination_skills.merge(df[['id_y', 'similarity', 'similarity_raw']], left_on='destination_id', right_on='id_y', how='left')
#     destination_skills.loc[destination_skills.similarity.isnull(), 'similarity'] = 0
#     destination_skills.loc[destination_skills.similarity.isnull(), 'similarity_raw'] = 0
    
#     # Extract the skill IDs and similarity values
#     skills_id = destination_skills.destination_id.to_list()
#     skills_id_sim = destination_skills.similarity.to_list()
#     skills_id_sim_raw = destination_skills.similarity_raw.to_list()  
    
#     # Save the skill IDs and similarity values
#     skills_id_ALL.append(skills_id)
#     skills_id_sim_ALL.append(skills_id_sim)
#     skills_id_raw_sim_ALL.append(skills_id_sim_raw)
    
#     ###### ORIGIN SKILLS ######
#     # Include unmatched skills as well
#     origin_skills = pd.DataFrame(data={'origin_id': occ_skills(job_i, ['Essential', 'Optional']).id.to_list()})
#     origin_skills = origin_skills.merge(df[['id_x', 'similarity', 'similarity_raw']], left_on='origin_id', right_on='id_x', how='left')
#     origin_skills.loc[origin_skills.similarity.isnull(), 'similarity'] = 0
#     origin_skills.loc[origin_skills.similarity.isnull(), 'similarity_raw'] = 0
    
#     # Extract the skill IDs and similarity values
#     skills_origin_id = origin_skills.origin_id.to_list()
#     skills_origin_id_sim = origin_skills.similarity.to_list()
#     skills_origin_id_sim_raw = origin_skills.similarity_raw.to_list()  
    
#     # Save the skill IDs and similarity values
#     skills_origin_id_ALL.append(skills_origin_id)
#     skills_origin_id_sim_ALL.append(skills_origin_id_sim)
#     skills_origin_id_raw_sim_ALL.append(skills_origin_id_sim_raw)    

#     ###### MATCHED SKILLS ######
#     # Include unmatched skills as well
#     matched_skills_origin.append(df.id_x.to_list())
#     matched_skills_destination.append(df.id_y.to_list())
#     matched_skills_similarity.append(df.similarity.to_list())
#     matched_skills_similarity_raw.append(df.similarity_raw.to_list())
 
    
# t_elapsed = time() - t 
# print(f'Time elapsed: {t_elapsed :.2f} sec ({t_elapsed/len(trans_to_analyse): .3f} per transition)')



Time elapsed: 178.16 sec ( 0.049 per transition)


In [33]:
# pickle.dump(skills_id_ALL, open(data_folder + 'interim/Transitions_Very_high_risk_ID.pickle','wb'))
# pickle.dump(skills_id_sim_ALL, open(data_folder + 'interim/Transitions_Very_high_risk_Sim.pickle','wb'))
# pickle.dump(skills_id_raw_sim_ALL, open(data_folder + 'interim/Transitions_Very_high_risk_Sim_Raw.pickle','wb'))

# pickle.dump(skills_origin_id_ALL, open(data_folder + 'interim/Transitions_Very_high_risk_origin_ID.pickle','wb'))
# pickle.dump(skills_origin_id_sim_ALL, open(data_folder + 'interim/Transitions_Very_high_risk_origin_Sim.pickle','wb'))
# pickle.dump(skills_origin_id_raw_sim_ALL, open(data_folder + 'interim/Transitions_Very_high_risk_origin_Sim_Raw.pickle','wb'))

# pickle.dump(matched_skills_origin, open(data_folder + 'interim/Transitions_Very_high_risk_matched_origin_ID.pickle','wb'))
# pickle.dump(matched_skills_destination, open(data_folder + 'interim/Transitions_Very_high_risk_matched_destination_ID.pickle','wb'))
# pickle.dump(matched_skills_similarity, open(data_folder + 'interim/Transitions_Very_high_risk_matched_Sim_Raw.pickle','wb'))
# pickle.dump(matched_skills_similarity_raw, open(data_folder + 'interim/Transitions_Very_high_risk_matched_Sim_Raw.pickle','wb'))


## Load the precomputed results

In [37]:
skills_id_ALL = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_ID.pickle','rb'))
skills_id_sim_ALL = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_Sim.pickle','rb'))
skills_id_raw_sim_ALL = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_Sim_Raw.pickle','rb'))

skills_origin_id_ALL = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_origin_ID.pickle','rb'))
skills_origin_id_sim_ALL = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_origin_Sim.pickle','rb'))
skills_origin_id_raw_sim_ALL = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_origin_Sim_Raw.pickle','rb'))

matched_skills_origin = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_matched_origin_ID.pickle','rb'))
matched_skills_destination = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_matched_destination_ID.pickle','rb'))
matched_skills_similarity = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_matched_Sim_Raw.pickle','rb'))
matched_skills_similarity_raw = pickle.load(open(data_folder + 'interim/Transitions_Very_high_risk_matched_Sim_Raw.pickle','rb'))



In [38]:
len(matched_skills_similarity_raw)

3619

# 2. Transition skills

In [173]:
def merge_lists(lists_to_merge, skills_type='destination'):
    
    """
    Dataframe with all the skills involved in the transitions 
    """
    
    # Merge lists
    list_skills = []
    list_similarity = []
    list_similarity_raw = []
    
    for i in lists_to_merge:        
        if skills_type=='destination':
            list_skills += skills_id_ALL[i]
            list_similarity += skills_id_sim_ALL[i]
            list_similarity_raw += skills_id_raw_sim_ALL[i]
        elif skills_type=='origin':
            list_skills +=  skills_origin_id_ALL[i]
            list_similarity += skills_origin_id_sim_ALL[i]
            list_similarity_raw += skills_origin_id_raw_sim_ALL[i]
            
    skill_similarities_all = pd.DataFrame(data={
        'skills_id': list_skills,
        'similarity': list_similarity,
        'similarity_raw': list_similarity_raw})
    
    # If a skill was not matched, then set it to 0
    skill_similarities_all.loc[skill_similarities_all.similarity_raw.isnull(), 'similarity_raw'] = 0
    
    return skill_similarities_all


def clean_up_df(df, n_trans):
    
    """ Clean up the dataframe """
    
    df['prevalence'] = np.round(df['counts'] / n_trans, 3)
    df.similarity = df.similarity.round(3) 
    df.reset_index(drop=True, inplace=True)    
    return df

def count_and_agg_scores(skill_similarities_all, groupby_column):
    
    """ Aggregates scores for each skill or cluster (depending on groupby_column) """
    
    # Counts
    x_counts = skill_similarities_all.groupby(groupby_column).count()
    # Mean similarity
    skill_similarities = skill_similarities_all.groupby(groupby_column).mean()
    # Create the dataframe
    skill_similarities['counts'] = x_counts['similarity']
    skill_similarities['stdev'] = skill_similarities_all.groupby(groupby_column).std()['similarity']
    skill_similarities.reset_index(inplace=True)
    return skill_similarities

def get_agg_matching_scores_Skills(skill_similarities_all, n_trans=1):

    """ Agregates scores for skills """
    
    # Aggregate scores
    skill_similarities = count_and_agg_scores(skill_similarities_all, 'skills_id')

    # Add information about skills
    skill_similarities = skill_similarities.merge(skills[['id', 'preferred_label',
                                                          'level_1', 'level_2', 'level_3', 'code']],
                                                  left_on='skills_id', right_on='id', how='left')
    
    # Clean up the dataframe
    skill_similarities = clean_up_df(skill_similarities, n_trans)
    skill_similarities = skill_similarities[['id', 'preferred_label', 'code', 'counts', 'prevalence', 'similarity' , 'stdev']]
    
    return skill_similarities

    
def get_agg_matching_scores_Clusters(skill_similarities_all, level='level_1', n_trans=1):
    
    """ Agregates scores for ESCO skills clusters """
    
    # Add skills cluster information
    skill_similarities_all_clust = skill_similarities_all.merge(skills[['id', 'preferred_label', 'level_1', 'level_2', 'level_3', 'code']], left_on='skills_id', right_on='id')
    
    # Aggregate scores
    skill_similarities = count_and_agg_scores(skill_similarities_all_clust, level)
    
    # Add skills cluster title
    skill_similarities = skill_similarities.merge(concepts[['code','title']], left_on=level, right_on='code')
    
    # Clean up the dataframe
    skill_similarities = clean_up_df(skill_similarities, n_trans)
    skill_similarities = skill_similarities[['code', 'title', 'counts', 'prevalence', 'similarity', 'stdev']]
        
    return skill_similarities

def get_stats_most_prevalent(skill_similarities, top_n):
    
    return skill_similarities.sort_values('prevalence', ascending=False).head(top_n).sort_values('similarity', ascending=True)



## 2.1 Skills for a single occupation

In [488]:
# Find an occupation
occ_risk[occ_risk.preferred_label.str.contains('human resources officer')]
occ_df = occ_risk[occ_risk.preferred_label.str.contains('hotel concierge')]
occ_df = occ_risk[occ_risk.preferred_label.str.contains('shop assistant')]
# occ_df = occ_risk[occ_risk.preferred_label.str.contains('sales assistant')]
occ_df

,id,concept_uri,preferred_label,isco_level_4,onet_code,onet_title,risk,prop_bottleneck_tasks,prop_phys_bottleneck_tasks,risk_std,bottleneck_quartile,risk_quartile,risk_cat,risk_cat_label,proportion_of_tasks
139,139,http://data.europa.eu/esco/occupation/0b15375e...,shop assistant,5223,41-2031.00,retail salespersons,3.578497,0.241128,0.137691,0.229566,0.0,3.0,0.0,Very high risk,0.542004


In [489]:
# Select the occupation
job_i = occ_df.id.values[0]

In [490]:
# Description of the occupation
occupations.loc[job_i].description

'Shop assistants work in shops where they perform assistance duties. The help shopkeepers in their daily work such as ordering and refilling of goods and stock, providing general advice to customers, selling products and maintaining the shop.'

In [491]:
# Show all transitions
all_trans = trans_to_analyse[trans_to_analyse.origin_id==job_i].copy()
all_trans = all_trans.merge(occ_clust[['id', 'preferred_label','level_1','level_2']], how='left', left_on='destination_id', right_on='id')
all_trans = all_trans[['origin_id','destination_id','preferred_label','similarity','level_1','level_2']]
all_trans

,origin_id,destination_id,preferred_label,similarity,level_1,level_2
0,139,2481,shelf filler,0.412046,2,14
1,139,2674,hotel butler,0.381527,2,11
2,139,2779,merchandiser,0.379778,2,13
3,139,1885,shop manager,0.379706,2,13
4,139,2168,supermarket manager,0.369951,2,13
5,139,1984,retail department manager,0.359325,2,13
6,139,2465,club host/club hostess,0.344195,2,11
7,139,463,cloak room attendant,0.334423,2,11
8,139,653,visual merchandiser,0.331160,2,15
9,139,2492,beauty salon attendant,0.327997,2,11


### 2.1.1 Transitions to one occupation

In [493]:
# Create the input dataframe in the required format
def trans_i_j(job_i, job_j):

    node_to_items_ = pd.concat([
        node_to_all_items.loc[[job_i]],
#         node_to_essential_items.loc[[job_i]],
        node_to_essential_items.loc[[job_j]]])

    # Compare jobs
    df, score = compare_nodes_utils.two_node_comparison(
            node_to_items_,
            job_i, job_j,
            skills[['id','preferred_label']],
            embeddings,
            metric='cosine',
            matching_method='one_to_one',
            symmetric=False)
    
    return df

In [524]:
job_j = all_trans.destination_id.iloc[2]
print(f'{occupations.loc[job_j].preferred_label} ({len(node_to_essential_items.loc[[job_j]].items_list.values[0])})')
print('---')
print(occupations.loc[job_j].description)
trans_df = trans_i_j(job_i, job_j)
trans_df

merchandiser (27)
---
Merchandisers are responsible for positioning goods following standards and procedures.


,id_x,preferred_label_x,item,id_y,preferred_label_y,similarity,similarity_raw
0,11989,supervise merchandise displays,0,11989,supervise merchandise displays,0.998,1.000000
1,1809,maintain relationship with customers,1,1809,maintain relationship with customers,0.998,1.000000
2,3248,maintain relationship with suppliers,2,3248,maintain relationship with suppliers,0.998,1.000000
3,6740,stock shelves,3,6740,stock shelves,0.998,1.000000
4,10034,keep records of merchandise delivery,4,10034,keep records of merchandise delivery,0.998,1.000000
5,6261,sales promotion techniques,5,12121,sales strategies,0.620,0.879821
6,2340,provide customer guidance on product selection,6,13369,negotiate buying conditions,0.321,0.855002
7,10543,carry out active selling,7,4325,merchandising techniques,0.237,0.846596
8,11706,provide customer follow-up services,8,10825,keep promotions records,0.229,0.845666
9,5971,put up price tags,9,5066,negotiate terms with suppliers,0.202,0.842544


In [ ]:
job_j = all_trans.destination_id.iloc[2]
print(f'{occupations.loc[job_j].preferred_label} ({len(node_to_essential_items.loc[[job_j]].items_list.values[0])})')
print('---')
print(occupations.loc[job_j].description)
trans_df = trans_i_j(job_i, job_j)
trans_df

In [578]:
data.skills.loc[6375].description

'Arrange goods in attractive and safe way. Set up a counter or other display area where demonstrations take place in order to attract the attention of prospective customers. Organise and maintain stands for merchandise display. Create and assemble sales spot and product displays for sales process.'

In [577]:
data.skills.loc[2122].description

'Place a particular item in more than one location within the store, in order to attract customer attention and increase sales.'

In [510]:
# Check the resilience of the transition destination
occ_n_trans[occ_n_trans.id.isin([job_j])].n_safe_viable_desirable

1549    13
Name: n_safe_viable_desirable, dtype: int64

In [500]:
trans_df_export = trans_df[['preferred_label_x', 'preferred_label_y', 'similarity']].copy()
trans_df_export.rename(columns={
    'preferred_label_x': 'origin skills',
    'preferred_label_y': 'destination skills'
}, inplace=True)
trans_df_export.to_csv(outputs_folder + f'skills_gap_Example_Shop_assistant_{job_i}_to_{job_j}.csv')


### 2.1.2 Across all transitions from the single occupation

In [435]:
# Lists of "transition skills" to merge into a dataframe
lists_to_merge = trans_to_analyse.reset_index()[trans_to_analyse.reset_index().origin_id==job_i].index

# Number of transitions we have
n_trans = len(lists_to_merge)

# SET UP which skills are we checking (destination vs origin)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')

get_stats_most_prevalent(
    get_agg_matching_scores_Skills(skill_similarities_all, n_trans),
    10)


,id,preferred_label,code,counts,prevalence,similarity,stdev
38,2052,greet guests,S3.4.4,4,0.20,0.000,0.000000
176,11012,have computer literacy,S5.5.0,4,0.20,0.000,0.000000
128,8440,maximise sales revenues,S1.6.1,4,0.20,0.003,0.003464
137,8831,ensure compliance with purchasing and contract...,S3.3.3,4,0.20,0.148,0.060000
32,1737,order supplies,S1.6.3,3,0.15,0.558,0.000000
14,710,communicate with customers,S3.4.1,5,0.25,0.802,0.000000
98,6288,maintain customer service,A1.12.3,9,0.45,0.998,0.000000
34,1809,maintain relationship with customers,S1.2.3,8,0.40,0.998,0.000000
86,5558,identify customer's needs,S1.7.2,7,0.35,0.998,0.000000
52,3248,maintain relationship with suppliers,S1.2.3,5,0.25,0.998,0.000000


In [436]:
get_stats_most_prevalent(
    get_agg_matching_scores_Clusters(skill_similarities_all, 'level_3', n_trans),
    10)


,code,title,counts,prevalence,similarity,stdev
85,S6.11.2,cleaning interior and exterior of buildings,10,0.50,0.015,0.027861
19,S1.1.1,negotiating and managing contracts and agreements,10,0.50,0.087,0.138132
62,S3.4.4,providing general assistance to people,14,0.70,0.130,0.328308
13,K4.1.6,wholesale and retail sales,9,0.45,0.161,0.335159
32,S1.6.1,selling products or services,11,0.55,0.173,0.336668
58,S3.3.3,complying with legal and organisational guidel...,17,0.85,0.300,0.402167
33,S1.6.2,"promoting products, services, or programs",11,0.55,0.422,0.425175
60,S3.4.1,providing information to the public and clients,9,0.45,0.446,0.421771
0,A1.12.3,provide high quality client service,13,0.65,0.970,0.068723
27,S1.2.3,developing professional relationships or networks,14,0.70,0.975,0.087662


## 2.2 Main results: Analyse the skills for all transitions

In [661]:
# Which lists do we need
lists_to_merge = range(len(trans_to_analyse))

# Number of transitions we have
n_trans = len(lists_to_merge)

# SET UP which skills are we checking (destination vs origin)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')

# Get skills scores
skill_similarities = get_agg_matching_scores_Skills(skill_similarities_all, n_trans)

# Rank
df = get_stats_most_prevalent(skill_similarities, 25) 

df.sort_values('similarity', ascending=False)

,id,preferred_label,code,counts,prevalence,similarity,stdev
1495,4223,perform scientific research,S2.1.1,200,0.055,0.677,0.464978
3049,8560,listen actively,S1.7.3,219,0.061,0.649,0.467640
253,710,communicate with customers,S3.4.1,187,0.052,0.635,0.411349
982,2778,manage staff,S4.8.1,654,0.181,0.616,0.433876
4421,12451,scientific research methodology,K1.0.1,188,0.052,0.615,0.483355
2333,6488,advise on financial matters,S1.5.2,204,0.056,0.505,0.473405
1646,4633,create a financial plan,S4.1.2,202,0.056,0.498,0.451084
1969,5558,identify customer's needs,S1.7.2,320,0.088,0.456,0.433621
621,1776,manage budgets,S4.3.1,663,0.183,0.433,0.488648
183,523,apply company policies,S3.3.3,203,0.056,0.414,0.383807


In [639]:
print(len(skill_similarities.prevalence)*0.01)
np.percentile(skill_similarities.prevalence,99)

48.1


0.03690999999999985

In [640]:
skill_similarities[skill_similarities.prevalence>0.03699].sort_values('similarity')

,id,preferred_label,code,counts,prevalence,similarity,stdev
3319,9325,analyse business requirements,S2.7.4,190,0.053,0.091,0.229146
2216,6202,budgetary principles,K4.1.2,159,0.044,0.140,0.327797
4214,11861,copyright legislation,K4.2.0,182,0.050,0.165,0.371272
3926,10988,present reports,S1.4.1,146,0.040,0.199,0.364536
833,2369,government policy implementation,K3.1.2,160,0.044,0.212,0.409501
2898,8108,business process modelling,K4.1.3,133,0.037,0.218,0.314621
4778,13387,perform market research,S2.1.1,187,0.052,0.229,0.392137
4044,11313,product comprehension,K4.1.6,162,0.045,0.229,0.402703
3506,9852,plan health and safety procedures,S4.1.5,197,0.054,0.233,0.404657
1372,3840,provide cost benefit analysis reports,S2.2.1,203,0.056,0.234,0.400614


In [625]:
df_all = get_stats_most_prevalent(
    get_agg_matching_scores_Clusters(skill_similarities_all, 'level_3', n_trans),
    20)
df_all[df_all.code.str.contains('S')]

,code,title,counts,prevalence,similarity,stdev
189,S2.7.4,analysing business operations,1348,0.372,0.120,0.274432
235,S4.1.3,developing operational policies and procedures,1327,0.367,0.160,0.289900
241,S4.2.1,directing operational activities,1143,0.316,0.162,0.318660
242,S4.2.2,planning and scheduling events and activities,1076,0.297,0.167,0.336927
100,S1.1.1,negotiating and managing contracts and agreements,769,0.212,0.169,0.334173
255,S4.8.3,monitoring and evaluating the performance of i...,829,0.229,0.229,0.370156
123,S1.2.1,coordinating activities with others,1830,0.506,0.237,0.399225
234,S4.1.2,"developing financial, business or marketing plans",956,0.264,0.240,0.381373
188,S2.7.3,analysing financial and economic data,901,0.249,0.249,0.384831
164,S2.1.1,conducting investigations,1209,0.334,0.289,0.438031


## 2.3 Analyse specific type of pathways (cluster-to-cluster)

In [743]:
def merge_clusters(base_df, id_col, risk_col_label):
    """
    Add risk info to the dataframe
    """
    trans_ = base_df.merge(data.occ_top[['id','level_1','level_2']],
                               left_on=id_col, right_on='id', how='left').rename(
        columns={
            'level_1': risk_col_label+'_level_1',
            'level_2': risk_col_label+'_level_2',
        }).drop('id', axis=1)
    
    return trans_

def get_transitions_from_to(trans_clust, from_level, from_cluster, to_level, to_cluster, filter_occ=None):
    trans_to_merge = trans_clust[(trans_clust['origin_'+from_level].isin([from_cluster])) & 
                                 (trans_clust['destination_'+to_level].isin([to_cluster]))]
    
    if type(filter_occ) != type(None):
        trans_to_merge = trans_to_merge[trans_to_merge.origin_id.isin(filter_occ)]
    
    return trans_to_merge, trans_to_merge.index.to_list()


In [699]:
trans_clust = merge_clusters(trans_to_analyse, 'origin_id', 'origin')
trans_clust = merge_clusters(trans_clust, 'destination_id', 'destination')
trans_clust.head(1)

,origin_id,origin_label,destination_id,destination_label,similarity,is_jobzone_ok,is_earnings_ok,is_safe,is_not_vhr,is_safer,...,W_activities,W_work_context,sim_category,is_viable_desirable,is_safe_destination,category,origin_level_1,origin_level_2,destination_level_1,destination_level_2
0,29,legal guardian,2867,recreational therapist,0.37022,True,True,True,True,True,...,0.536452,0.731929,min_viable,True,True,min_viable,13,52,10,41


In [1092]:
# Find occupations that participate in both pathways
set1 = trans_clust[(trans_clust.origin_level_1==2) &
            (trans_clust.destination_level_1==2)].origin_id.unique()

set2 = trans_clust[(trans_clust.origin_level_1==2) &
            (trans_clust.destination_level_1==1)].origin_id.unique()

transitions_to_both = list(set(set2).intersection(set(set1)))
print(len(transitions_to_both) / len(trans_clust[(trans_clust.origin_level_1==2)].origin_id.unique()))

0.5217391304347826


Approx. 50% of high risk occupations from the sales and services cluster have transitions to both sales and services and business admin clusters.

In [1088]:
# data.occ_top[data.occ_top.id.isin(transitions_to_both)]

In [776]:
from scipy.stats import kruskal, wilcoxon
from scipy.stats import percentileofscore

def merge_level_titles(df_new_transitions):
    df_new_transitions_code = df_new_transitions.merge(data.skills[['id','skill_category', 'level_1','level_2']],
                                                       left_on='id', right_on='id', how='left')
    df_new_transitions_code = df_new_transitions_code.merge(data.concepts[['code','title']], how='left')
    df_new_transitions_code.rename(columns={'title' : 'title_level_3', 'skill_category': 'category'}, inplace=True)
    
    df_new_transitions_code = df_new_transitions_code.merge(data.concepts[['code', 'title']],
                                                           left_on='level_2', right_on='code', how='left')
    df_new_transitions_code.rename(columns={'title' : 'title_level_2'}, inplace=True)
    
    df_new_transitions_code = df_new_transitions_code.merge(data.concepts[['code', 'title']],
                                                           left_on='level_1', right_on='code', how='left')
    df_new_transitions_code.rename(columns={'title' : 'title_level_1'}, inplace=True)
    
    return df_new_transitions_code

### 2.3.1 From *sales and services* to *business admin*

In [1064]:
# Check all at-risk occupations (without any filtering)
trans_to_merge, lists_to_merge = get_transitions_from_to(trans_clust, 
                                                         from_level='level_1', from_cluster=2, 
                                                         to_level='level_1', to_cluster=1,
                                                        )

# Number of transitions we have
n_trans = len(lists_to_merge)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')
skill_similarities = get_agg_matching_scores_Skills(skill_similarities_all, n_trans)
df = get_stats_most_prevalent(skill_similarities, 15) 
df

,id,preferred_label,code,counts,prevalence,similarity,stdev
262,5472,corporate social responsibility,K4.1.3,26,0.193,0.000,0.000000
470,9852,plan health and safety procedures,S4.1.5,19,0.141,0.000,0.000918
577,12026,train employees,S1.3.3,16,0.119,0.002,0.002733
596,12672,liaise with managers,S1.2.1,21,0.156,0.034,0.043252
604,12932,follow company standards,S3.3.3,26,0.193,0.046,0.064947
510,10770,plan medium to long term objectives,S4.1.0,20,0.148,0.250,0.443373
480,10065,develop professional network,S1.2.3,28,0.207,0.256,0.285208
521,10988,present reports,S1.4.1,22,0.163,0.262,0.384232
6,160,create solutions to problems,S1.9.1,28,0.207,0.345,0.459236
379,7794,manage contracts,S1.1.1,19,0.141,0.409,0.464426


In [1075]:
# Most common destination occupations
trans_to_merge.groupby('destination_label').count().sort_values('destination_id').tail(15)

,origin_id,origin_label,destination_id,similarity,is_jobzone_ok,is_earnings_ok,is_safe,is_not_vhr,is_safer,job_zone_dif,...,W_activities,W_work_context,sim_category,is_viable_desirable,is_safe_destination,category,origin_level_1,origin_level_2,destination_level_1,destination_level_2
destination_label,,,,,,,,,,,,,,,,,,,,,
costume buyer,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
court administrator,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
civil service administrative officer,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
consumer rights advisor,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
producer,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
debt collector,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
energy manager,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
advertising specialist,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
ICT product manager,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [1070]:
# Most common destination clusters
trans_to_merge.groupby('destination_level_2').count().sort_values('destination_id').tail(15)

,origin_id,origin_label,destination_id,destination_label,similarity,is_jobzone_ok,is_earnings_ok,is_safe,is_not_vhr,is_safer,...,W_skills,W_activities,W_work_context,sim_category,is_viable_desirable,is_safe_destination,category,origin_level_1,origin_level_2,destination_level_1
destination_level_2,,,,,,,,,,,,,,,,,,,,,
7,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
8,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
9,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
6,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,53,53,53,53,53


In [1073]:
# Cluster legend
data.clusters_level_2[data.clusters_level_2.level_1==1]

,level_1,level_2,keywords,keywords_manual,comments,comments_risk,counts
6,1,6,"manager, supervisor, assembly supervisor, assembly, facility manager, facility, production, production supervisor, centre, director",project & process managers,"very diverse, at the core of the map",could be broken down into more industrial & business roles,127
7,1,7,"financial, insurance, analyst, manager, broker, trader, investment, credit, estate, real estate",accounting & financial services workers,includes real estate,-,98
8,1,8,"assistant, clerk, officer, court, administrative, office, legal, administrator, survey, assistant tax","assistants, clerks & legal workers",-,could be broken down into clerks & legal workers,75
9,1,9,"manager, business, product manager, officer, policy, manager ict, analyst, ict, advertising, chief",business managers,-,-,62
10,1,10,"policy officer, policy, officer, affair, consultant, political, development, programme, social, public",public officials,-,-,57


In [1039]:
# Check the most prevalent skills, i.e. the 95th percentile

# print(len(skill_similarities.prevalence)*0.01)
# scipy.stats import percentileofscore
thresh_prev = np.percentile(skill_similarities.prevalence, 95)
thresh_prev

0.081

In [1040]:
# Most prevalent skills and their matching scores
most_prev = skill_similarities[skill_similarities.prevalence>thresh_prev].sort_values('similarity')
most_prev

,id,preferred_label,code,counts,prevalence,similarity,stdev
262,5472,corporate social responsibility,K4.1.3,26,0.193,0.000,0.000000
470,9852,plan health and safety procedures,S4.1.5,19,0.141,0.000,0.000918
577,12026,train employees,S1.3.3,16,0.119,0.002,0.002733
370,7661,supervise daily information operations,S4.2.1,12,0.089,0.027,0.043147
596,12672,liaise with managers,S1.2.1,21,0.156,0.034,0.043252
604,12932,follow company standards,S3.3.3,26,0.193,0.046,0.064947
449,9413,ensure cross-department cooperation,S1.2.1,13,0.096,0.083,0.153395
114,2263,strive for company growth,S4.1.2,12,0.089,0.112,0.081358
541,11286,analyse data about clients,S4.8.3,14,0.104,0.140,0.240941
178,3651,manage health and safety standards,S3.3.1,15,0.111,0.147,0.346937


In [1041]:
# wilcoxon(skill_similarities_all[skill_similarities_all.skills_id==12672].similarity.values)

In [1042]:
# Total matching to all destination skills, weighted by the prevalence
np.sum(skill_similarities['prevalence'] * skill_similarities['similarity'])/np.sum(skill_similarities.prevalence)

0.17607902841536405

In [1043]:
# Add skill hierarchy categories (skill clusters)
skill_similarities_clust = merge_level_titles(skill_similarities)
skill_similarities_clust.head(2)

,id,preferred_label,code_x,counts,prevalence,similarity,stdev,category,level_1,level_2,title_level_3,code_y,title_level_2,code,title_level_1
0,8,apply diplomatic principles,S1.1.2,1,0.007,0.000,NaN,S,S1,S1.1,mediating and resolving disputes,S1.1,negotiating,S1,"communication, collaboration and creativity"
1,31,guarantee customer satisfaction,A1.12.3,9,0.067,0.796,0.224983,A,A1,A1.12,provide high quality client service,A1.12,manage quality,A1,attitudes


In [1044]:
most_prev = most_prev[['preferred_label', 'counts', 'prevalence', 'similarity']].copy().sort_values('similarity', ascending=False)
most_prev = most_prev.to_csv(outputs_folder + f'skills_gap_Example_SalesServices_to_BusinessAdmin.csv')


In [542]:
for j, row in df.iterrows():
    print(f"({row.similarity:.2f}) {row.preferred_label}")

(0.00) corporate social responsibility
(0.00) plan health and safety procedures
(0.00) train employees
(0.03) liaise with managers
(0.05) follow company standards
(0.25) plan medium to long term objectives
(0.26) develop professional network
(0.26) present reports
(0.34) create solutions to problems
(0.41) manage contracts
(0.45) manage staff
(0.49) manage budgets
(0.50) handle customer complaints
(0.62) comply with food safety and hygiene
(0.69) identify customer's needs


### Analysis of the most prevalent, third-level skill categories

In [978]:
cluster_similarities = get_agg_matching_scores_Clusters(skill_similarities_all, 'level_3', n_trans)
df = get_stats_most_prevalent(cluster_similarities, 15) 
df

,code,title,counts,prevalence,similarity,stdev
33,K4.2.0,law,62,0.459,0.035,0.177484
27,K4.1.2,"finance, banking and insurance",45,0.333,0.035,0.150415
28,K4.1.3,management and administration,94,0.696,0.042,0.123485
133,S4.2.1,directing operational activities,84,0.622,0.086,0.219018
117,S3.3.3,complying with legal and organisational guidelines,46,0.341,0.103,0.210724
128,S4.1.3,developing operational policies and procedures,57,0.422,0.113,0.240418
134,S4.2.2,planning and scheduling events and activities,84,0.622,0.120,0.271697
106,S2.7.4,analysing business operations,70,0.519,0.126,0.303496
29,K4.1.4,marketing and advertising,67,0.496,0.126,0.263639
127,S4.1.2,"developing financial, business or marketing plans",48,0.356,0.170,0.308652


In [992]:
# Take the most prevalent (90th percentile) skill clusteers
clust_prev = np.percentile(cluster_similarities.prevalence, 90)
clust_prev = cluster_similarities[cluster_similarities.prevalence > clust_prev].sort_values('similarity')

In [993]:
# Show also the most prevalent skills within each cluster
x = []
for j, row in clust_prev.iterrows():
    dff = skill_similarities_clust[skill_similarities_clust.code_x==row.code]
    dff = dff.sort_values('prevalence', ascending=False).iloc[0:3]
    xx = []
    for jj, rrow in dff.iterrows():
        xx.append(f'{rrow.preferred_label} ({np.round(rrow.similarity,2)})')
    x.append(', '.join(xx))
    
    

In [994]:
clust_prev_ = clust_prev.copy()
clust_prev_['skills'] = x
clust_prev_

,code,title,counts,prevalence,similarity,stdev,skills
27,K4.1.2,"finance, banking and insurance",45,0.333,0.035,0.150415,"budgetary principles (0.0), debt collection techniques (0.0), public finance (0.01)"
33,K4.2.0,law,62,0.459,0.035,0.177484,"contract law (0.17), employment law (0.0), copyright legislation (0.0)"
28,K4.1.3,management and administration,94,0.696,0.042,0.123485,"corporate social responsibility (0.0), project management (0.02), cost management (0.04)"
133,S4.2.1,directing operational activities,84,0.622,0.086,0.219018,"supervise daily information operations (0.03), perform project management (0.03), supervise work (0.04)"
117,S3.3.3,complying with legal and organisational guidelines,46,0.341,0.103,0.210724,"follow company standards (0.05), adhere to organisational guidelines (0.32), ensure lawful business operations (0.0)"
128,S4.1.3,developing operational policies and procedures,57,0.422,0.113,0.240418,"develop strategies for accessibility (0.2), improve business processes (0.06), define technical requirements (0.32)"
134,S4.2.2,planning and scheduling events and activities,84,0.622,0.120,0.271697,"plan events (0.25), coordinate events (0.09), fix meetings (0.0)"
29,K4.1.4,marketing and advertising,67,0.496,0.126,0.263639,"advertising techniques (0.21), ICT market (0.0), marketing principles (0.53)"
106,S2.7.4,analysing business operations,70,0.519,0.126,0.303496,"measure customer feedback (0.64), analyse business objectives (0.0), analyse supply chain trends (0.09)"
127,S4.1.2,"developing financial, business or marketing plans",48,0.356,0.170,0.308652,"strive for company growth (0.11), develop business case (0.02), forecast occupancy demand (0.0)"


In [995]:
# Export the cluster results
clust_prev_.to_csv(outputs_folder + f'skills_gap_Example_SalesServices_to_BusinessAdmin_Cluster.csv')

### 2.3.2 From *sales & services* to *sales & services* clusters

In [1081]:
trans_to_merge, lists_to_merge = get_transitions_from_to(trans_clust, 
                                                         from_level='level_1', from_cluster=2, 
                                                         to_level='level_1', to_cluster=2,
#                                                          filter_occ=transitions_to_both                                                        
                                                        )

# Number of transitions we have
n_trans = len(lists_to_merge)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')
skill_similarities = get_agg_matching_scores_Skills(skill_similarities_all, n_trans)
df = get_stats_most_prevalent(skill_similarities, 15)
df

,id,preferred_label,code,counts,prevalence,similarity,stdev
335,8440,maximise sales revenues,S1.6.1,65,0.187,0.133,0.327407
77,1776,manage budgets,S4.3.1,62,0.178,0.214,0.407695
118,2778,manage staff,S4.8.1,85,0.244,0.266,0.347582
96,2187,handle customer complaints,S1.1.3,75,0.216,0.361,0.481162
460,11012,have computer literacy,S5.5.0,79,0.227,0.417,0.495332
471,11313,product comprehension,K4.1.6,72,0.207,0.438,0.483611
47,1127,characteristics of products,K7.2.2,78,0.224,0.450,0.479946
560,13210,characteristics of services,K4.1.6,78,0.224,0.463,0.473276
402,9777,implement sales strategies,S1.6.2,62,0.178,0.590,0.445233
125,3248,maintain relationship with suppliers,S1.2.3,64,0.184,0.667,0.374838


In [1080]:
data.clusters_level_2[data.clusters_level_2.level_1==2]

,level_1,level_2,keywords,keywords_manual,comments,comments_risk,counts
11,2,11,"attendant, cleaner, chef, restaurant, head, room attendant, cook, steward, tourist, stewardess","food, cleaning & customer services workers","includes food, cleaning, hospitality & beauty","could be broken down into hospitality, cleaning, beauty, food",97
12,2,12,"representative, sale, sale representative, service representative, rental, rental service, technical sale, service, technical, machinery",customer representatives,-,-,62
13,2,13,"shop manager, shop, manager, equipment shop, equipment, station manager, retail, manager casino, manager building, accessory shop",shop & services managers,-,-,61
14,2,14,"specialised seller, seller, specialised, equipment specialised, accessory specialised, accessory, equipment, seller computer, seller domestic, good specialised",retail & sales workers,"includes personal shoppers, stylists",-,54
15,2,15,"merchant, wholesale merchant, wholesale, product wholesale, equipment, machinery, manager wholesale, equipment wholesale, product, merchant agricultural",wholesale merchants,-,-,48


In [1077]:
trans_to_merge.groupby('destination_label').count().sort_values('destination_id').tail(15)

,origin_id,origin_label,destination_id,similarity,is_jobzone_ok,is_earnings_ok,is_safe,is_not_vhr,is_safer,job_zone_dif,...,W_activities,W_work_context,sim_category,is_viable_desirable,is_safe_destination,category,origin_level_1,origin_level_2,destination_level_1,destination_level_2
destination_label,,,,,,,,,,,,,,,,,,,,,
technical sales representative in electronic and telecommunications equipment,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
pawnbroker,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
sales processor,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
supermarket manager,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
ICT account manager,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
locker room attendant,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
retail department manager,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
lottery operator,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
ICT help desk manager,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [1079]:
trans_to_merge.groupby('destination_level_2').count().sort_values('destination_id').tail(15)

,origin_id,origin_label,destination_id,destination_label,similarity,is_jobzone_ok,is_earnings_ok,is_safe,is_not_vhr,is_safer,...,W_skills,W_activities,W_work_context,sim_category,is_viable_desirable,is_safe_destination,category,origin_level_1,origin_level_2,destination_level_1
destination_level_2,,,,,,,,,,,,,,,,,,,,,
14,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
15,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
13,79,79,79,79,79,79,79,79,79,79,...,79,79,79,79,79,79,79,79,79,79
12,109,109,109,109,109,109,109,109,109,109,...,109,109,109,109,109,109,109,109,109,109
11,111,111,111,111,111,111,111,111,111,111,...,111,111,111,111,111,111,111,111,111,111


In [1085]:
# print(len(skill_similarities.prevalence)*0.01)
# scipy.stats import percentileofscore
thresh_prev = np.percentile(skill_similarities.prevalence,95)
thresh_prev

0.10719999999999959

In [1047]:
most_prev = skill_similarities[skill_similarities.prevalence>thresh_prev].sort_values('similarity')
most_prev

,id,preferred_label,code,counts,prevalence,similarity,stdev
490,11856,control of expenses,S2.8.3,41,0.118,0.005,0.005010
360,8831,ensure compliance with purchasing and contracting regulations,S3.3.3,54,0.155,0.091,0.194338
151,3879,recruit employees,S4.7.0,41,0.118,0.126,0.329484
335,8440,maximise sales revenues,S1.6.1,65,0.187,0.133,0.327407
77,1776,manage budgets,S4.3.1,62,0.178,0.214,0.407695
118,2778,manage staff,S4.8.1,85,0.244,0.266,0.347582
43,1024,ensure customer focus,S4.1.0,39,0.112,0.282,0.454953
8,160,create solutions to problems,S1.9.1,48,0.138,0.343,0.448444
229,6039,negotiate sales contracts,S1.1.1,38,0.109,0.351,0.442543
96,2187,handle customer complaints,S1.1.3,75,0.216,0.361,0.481162


In [1048]:
np.sum(df['prevalence'] * df['similarity'])/np.sum(df.prevalence)

0.555109011627907

In [1087]:
skill_similarities_clust = merge_level_titles(skill_similarities)
skill_similarities_clust[skill_similarities_clust.prevalence>thresh_prev].sort_values('similarity')

,id,preferred_label,code_x,counts,prevalence,similarity,stdev,category,level_1,level_2,title_level_3,code_y,title_level_2,code,title_level_1
490,11856,control of expenses,S2.8.3,41,0.118,0.005,0.005010,S,S2,S2.8,monitoring financial and economic resources and activity,S2.8,"monitoring, inspecting and testing",S2,information skills
360,8831,ensure compliance with purchasing and contracting regulations,S3.3.3,54,0.155,0.091,0.194338,S,S3,S3.3,complying with legal and organisational guidelines,S3.3,protecting and enforcing,S3,assisting and caring
151,3879,recruit employees,S4.7.0,41,0.118,0.126,0.329484,S,S4,S4.7,recruiting and hiring,S4.7,recruiting and hiring,S4,management skills
335,8440,maximise sales revenues,S1.6.1,65,0.187,0.133,0.327407,S,S1,S1.6,selling products or services,S1.6,"promoting, selling and purchasing",S1,"communication, collaboration and creativity"
77,1776,manage budgets,S4.3.1,62,0.178,0.214,0.407695,S,S4,S4.3,managing budgets or finances,S4.3,allocating and controlling resources,S4,management skills
118,2778,manage staff,S4.8.1,85,0.244,0.266,0.347582,S,S4,S4.8,supervising a team or group,S4.8,supervising people,S4,management skills
43,1024,ensure customer focus,S4.1.0,39,0.112,0.282,0.454953,S,S4,S4.1,developing objectives and strategies,S4.1,developing objectives and strategies,S4,management skills
8,160,create solutions to problems,S1.9.1,48,0.138,0.343,0.448444,S,S1,S1.9,developing solutions,S1.9,solving problems,S1,"communication, collaboration and creativity"
229,6039,negotiate sales contracts,S1.1.1,38,0.109,0.351,0.442543,S,S1,S1.1,negotiating and managing contracts and agreements,S1.1,negotiating,S1,"communication, collaboration and creativity"
96,2187,handle customer complaints,S1.1.3,75,0.216,0.361,0.481162,S,S1,S1.1,responding to complaints,S1.1,negotiating,S1,"communication, collaboration and creativity"


In [1050]:
# trans_to_merge.groupby('destination_level_2').count()

In [1051]:
# data.clusters_level_2[data.clusters_level_2.level_1==2]

In [1052]:
# skill_similarities_all

In [1053]:
np.sum(df['prevalence'] * df['similarity'])/np.sum(df.prevalence)

0.555109011627907

In [1054]:
most_prev = most_prev[['preferred_label', 'counts', 'prevalence', 'similarity']].copy().sort_values('similarity', ascending=False) 
most_prev.to_csv(outputs_folder + f'skills_gap_Example_SalesServices_to_SalesServices.csv')


### Analysis of the most prevalent, third-level skill categories

In [1055]:
pd.set_option('max_colwidth', 200)
cluster_similarities = get_agg_matching_scores_Clusters(skill_similarities_all, 'level_3', n_trans)
df = get_stats_most_prevalent(cluster_similarities, 15) 
df

,code,title,counts,prevalence,similarity,stdev
120,S4.2.1,directing operational activities,110,0.316,0.091,0.252502
121,S4.2.2,planning and scheduling events and activities,114,0.328,0.117,0.254498
103,S3.3.3,complying with legal and organisational guidelines,217,0.624,0.159,0.303401
69,S1.6.3,purchasing goods or services,109,0.313,0.188,0.262429
129,S4.8.1,supervising a team or group,115,0.330,0.200,0.319257
43,S1.1.1,negotiating and managing contracts and agreements,172,0.494,0.211,0.337594
67,S1.6.1,selling products or services,159,0.457,0.224,0.379117
110,S3.4.4,providing general assistance to people,231,0.664,0.276,0.426300
102,S3.3.1,complying with health and safety procedures,108,0.310,0.276,0.425201
26,K4.1.6,wholesale and retail sales,237,0.681,0.360,0.448918


In [1060]:
clust_prev = np.percentile(cluster_similarities.prevalence, 90)
clust_prev = cluster_similarities[cluster_similarities.prevalence > clust_prev].sort_values('similarity')

In [1061]:
x = []
for j, row in clust_prev.iterrows():
    dff = skill_similarities_clust[skill_similarities_clust.code_x==row.code]
    dff = dff.sort_values('prevalence', ascending=False).iloc[0:3]
    xx = []
    for jj, rrow in dff.iterrows():
        xx.append(f'{rrow.preferred_label} ({np.round(rrow.similarity,2)})')
    x.append(', '.join(xx))
    
    

In [1062]:
clust_prev_ = clust_prev.copy()
clust_prev_['skills'] = x
clust_prev_

,code,title,counts,prevalence,similarity,stdev,skills
120,S4.2.1,directing operational activities,110,0.316,0.091,0.252502,"oversee promotional sales prices (0.0), supervise sales activities (0.23), manage tender processes (0.0)"
121,S4.2.2,planning and scheduling events and activities,114,0.328,0.117,0.254498,"manage schedule of tasks (0.01), prioritise requests (0.27), prioritise tasks (0.15)"
28,K4.2.0,law,106,0.305,0.121,0.320278,"employment law (0.0), embargo regulations (0.0), legal standards in gambling (0.39)"
103,S3.3.3,complying with legal and organisational guidelines,217,0.624,0.159,0.303401,"ensure compliance with purchasing and contracting regulations (0.09), enforce regulations of selling alcoholic beverages to minors (0.09), apply company policies (0.14)"
69,S1.6.3,purchasing goods or services,109,0.313,0.188,0.262429,"order supplies (0.44), apply procurement (0.12), coordinate purchasing activities (0.02)"
129,S4.8.1,supervising a team or group,115,0.330,0.200,0.319257,"manage staff (0.27), supervise crew (0.0), lead a team (0.06)"
43,S1.1.1,negotiating and managing contracts and agreements,172,0.494,0.211,0.337594,"negotiate sales contracts (0.35), negotiate buying conditions (0.24), manage contracts (0.29)"
67,S1.6.1,selling products or services,159,0.457,0.224,0.379117,"maximise sales revenues (0.13), set sales promotions (0.0), work independently in sales (0.0)"
102,S3.3.1,complying with health and safety procedures,108,0.310,0.276,0.425201,"comply with food safety and hygiene (0.45), apply health and safety standards (0.28), manage health and safety standards (0.16)"
110,S3.4.4,providing general assistance to people,231,0.664,0.276,0.426300,"greet guests (0.38), assist customers (0.62), assist guest departure (0.0)"


In [965]:
# skill_similarities_clust[skill_similarities_clust.level_2.isin(clust_prev_.code.to_list())]

In [1020]:
clust_prev_.to_csv(outputs_folder + f'skills_gap_Example_SalesServices_to_SalesServices_Cluster.csv')

In [810]:
# skill_similarities.code

0      A1.12.3
1       S3.4.1
2       A1.3.0
3       S2.1.1
4       S2.9.0
        ...   
562    A1.16.0
563     S1.5.3
564     S1.1.1
565     S2.1.1
566     S4.1.2
Name: code, Length: 567, dtype: object

### 2.3.3 From clerical business & admin workers to everywhere else

In [1098]:
clerical_occ = data.occ_top[data.occ_top.isco_level_1==4].id.to_list()

In [1139]:
occ.loc[trans_clust[(trans_clust.origin_level_1==1) & trans_clust.origin_id.isin(clerical_occ)].origin_id.unique()].preferred_label.to_list()


['telephone switchboard operator',
 'machinery assembly coordinator',
 'auditing clerk',
 'bank teller',
 'freight transport dispatcher',
 'secretary',
 'sales support assistant',
 'taxi controller',
 'bus route supervisor',
 'office clerk',
 'tax clerk',
 'water traffic coordinator',
 'file clerk',
 'back office specialist',
 'foreign exchange cashier',
 'road transport maintenance scheduler',
 'billing clerk',
 'ship pilot dispatcher',
 'property assistant',
 'baggage flow supervisor',
 'tram controller',
 'aircraft cargo operations coordinator',
 'move coordinator',
 'receptionist',
 'port coordinator',
 'investment clerk']

In [1147]:
# Check all at-risk occupations (without any filtering)
trans_to_merge = trans_clust[(trans_clust.origin_level_1==1) & (trans_clust.origin_id.isin(clerical_occ)==True)]
lists_to_merge = trans_to_merge.index.to_list()

# Number of transitions we have
n_trans = len(lists_to_merge)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')
skill_similarities = get_agg_matching_scores_Skills(skill_similarities_all, n_trans)

In [1151]:
thresh_prev = np.percentile(skill_similarities.prevalence,95)
most_prev = skill_similarities[skill_similarities.prevalence>thresh_prev].sort_values('similarity')
most_prev = most_prev[['preferred_label', 'counts', 'prevalence', 'similarity']].copy().sort_values('similarity', ascending=False) 
most_prev.to_csv(outputs_folder + f'skills_gap_Example_BusinessAdminClerical_to_All.csv')

# most_prev

,preferred_label,counts,prevalence,similarity
990,handle financial transactions,15,0.069,0.931
608,financial analysis,13,0.060,0.818
681,perform clerical duties,17,0.078,0.674
199,manage staff,61,0.281,0.671
201,supervise unloading of cargo,13,0.060,0.664
230,supervise loading of cargo,13,0.060,0.654
548,quality standards,14,0.065,0.645
832,have computer literacy,19,0.088,0.630
284,financial statements,12,0.055,0.582
43,communicate with customers,26,0.120,0.569


In [1198]:
data.occ[data.occ.isco_level_4==2411]

,id,concept_uri,preferred_label,isco_level_4,onet_code,onet_title,risk,prop_bottleneck_tasks,prop_phys_bottleneck_tasks,risk_std,...,is_top_level,annual_gross_pay,total_paid_hours,employment_count,employment_rate,level_1,level_2,remote_labor_index,covid_score,physical_proximity
328,328,http://data.europa.eu/esco/occupation/1ad59946-d13e-441e-9b36-56e1df055e21,financial auditor,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
933,933,http://data.europa.eu/esco/occupation/4e4a291d-1d38-4c5a-812a-5827d0691b11,budget analyst,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
1075,1075,http://data.europa.eu/esco/occupation/59c22779-4d89-4bc9-ba8b-d64bea236723,tax advisor,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
1131,1131,http://data.europa.eu/esco/occupation/5e5b1b61-1351-4d3b-a293-deb3661715dd,financial controller,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
1455,1455,http://data.europa.eu/esco/occupation/78db356f-a96a-4456-ab29-2de62e52f830,public finance accountant,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
1589,1589,http://data.europa.eu/esco/occupation/84871c2a-abd1-45eb-b03a-65e51a26fd84,cost analyst,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
1644,1644,http://data.europa.eu/esco/occupation/89330b57-6a30-40e1-b623-50f47a4b0c34,grants management officer,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,8,0.8,0.279285,0.39
1896,1896,http://data.europa.eu/esco/occupation/a04eedcf-8424-475a-8dc5-f30a9da4b151,financial fraud examiner,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
2397,2397,http://data.europa.eu/esco/occupation/caafca69-9faa-4850-8930-a3fa07fea4e6,accounting analyst,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39
2422,2422,http://data.europa.eu/esco/occupation/cd4f41a5-87a6-4b3c-8ee3-14e6f457c365,audit supervisor,2411,13-2011.01,accountants,3.582878,0.299813,0.173454,0.288527,...,False,32466.699499,32.888501,NaN,NaN,1,7,0.8,0.279285,0.39


### Cluster analysis

In [1152]:
pd.set_option('max_colwidth', 200)
cluster_similarities = get_agg_matching_scores_Clusters(skill_similarities_all, 'level_3', n_trans)
clust_prev = np.percentile(cluster_similarities.prevalence, 90)
clust_prev = cluster_similarities[cluster_similarities.prevalence > clust_prev].sort_values('similarity')

skill_similarities_clust = merge_level_titles(skill_similarities)
x = []
for j, row in clust_prev.iterrows():
    dff = skill_similarities_clust[skill_similarities_clust.code_x==row.code]
    dff = dff.sort_values('prevalence', ascending=False).iloc[0:3]
    xx = []
    for jj, rrow in dff.iterrows():
        xx.append(f'{rrow.preferred_label} ({np.round(rrow.similarity,2)})')
    x.append(', '.join(xx))
    
clust_prev_ = clust_prev.copy()
clust_prev_['skills'] = x
clust_prev_    

,code,title,counts,prevalence,similarity,stdev,skills
33,K4.1.3,management and administration,97,0.447,0.064,0.184380,"corporate social responsibility (0.0), customer relationship management (0.07), financial management (0.2)"
155,S4.1.3,developing operational policies and procedures,56,0.258,0.075,0.179919,"ensure compliance with policies (0.03), consider constraints in maritime shipping (0.01), develop efficiency plans for logistics operations (0.25)"
127,S2.7.4,analysing business operations,64,0.295,0.080,0.249290,"measure customer feedback (0.01), assess the feasibility of implementing developments (0.0), analyse business plans (0.0)"
160,S4.2.2,planning and scheduling events and activities,149,0.687,0.093,0.262788,"fix meetings (0.02), forecast workload (0.0), administer appointments (0.22)"
38,K4.2.0,law,94,0.433,0.100,0.279700,"international regulations for cargo handling (0.1), legislation procedure (0.0), employment law (0.0)"
162,S4.3.1,managing budgets or finances,66,0.304,0.103,0.261590,"manage budgets (0.03), enforce financial policies (0.33), manage accounts (0.22)"
143,S3.3.3,complying with legal and organisational guidelines,144,0.664,0.182,0.347604,"follow company standards (0.2), apply company policies (0.34), apply airport standards and regulations (0.78)"
159,S4.2.1,directing operational activities,185,0.853,0.185,0.350266,"supervise work (0.02), supervise loading of cargo (0.65), perform project management (0.06)"
141,S3.3.1,complying with health and safety procedures,56,0.258,0.195,0.333419,"manage health and safety standards (0.04), apply safety management (0.31), implement airside safety procedures (0.4)"
36,K4.1.6,wholesale and retail sales,63,0.290,0.210,0.405036,"characteristics of services (0.0), quality standards (0.64), product comprehension (0.0)"


In [1197]:
# data.skills[data.skills.preferred_label.str.contains('manage staff')].id

In [1196]:
# from scipy.spatial.distance import cosine, cdist
# d = cdist(embeddings[2778,:].reshape(1,-1), embeddings, metric='cosine')
# xx = np.squeeze(np.sort(d))
# x = np.squeeze(np.argsort(d))

# 3. Case studies

In [340]:
# Import data
data_folder = '../../data/'
outputs_folder = '../../reports/outputs/'
data = load_data.Data()
sim = load_data.Similarities()

In [385]:
# Add covid scores to transition dataframe
trans_covid = trans_clust.merge(data.occ_top[['id' ,'covid_score']], left_on='origin_id', right_on='id', how='left')
trans_covid.rename(columns={'covid_score': 'origin_covid'}, inplace=True)
trans_covid.drop('id', axis=1, inplace=True)

trans_covid = trans_covid.merge(data.occ_top[['id' ,'covid_score']], left_on='destination_id', right_on='id', how='left')
trans_covid.rename(columns={'covid_score': 'destination_covid'}, inplace=True)
trans_covid.drop('id', axis=1, inplace=True)

## 3.0 Hotel concierge to different clusters

In [445]:
# Find an occupation
df = occ_risk[occ_risk.preferred_label.str.contains('hotel concierge')]
#df = occ_risk[occ_risk.preferred_label.str.contains('shop assistant')]
job_i = df.id.values[0]

In [446]:
data.occ.loc[job_i].covid_score

0.7478540727926137

In [447]:
trans_clust[trans_clust.origin_id==job_i].groupby('destination_level_1').agg({'destination_id': 'count'})

,destination_id
destination_level_1,
0,1
1,6
2,10
5,1


### 3.0.1 Hotel concierge to business admin clusters

In [441]:
# Lists of "transition skills" to merge into a dataframe
trans_to_merge = trans_clust[trans_clust.origin_id==job_i]
trans_to_merge = trans_to_merge[trans_to_merge.destination_level_1==2]
lists_to_merge = trans_to_merge.index

# Number of transitions we have
n_trans = len(lists_to_merge)

# SET UP which skills are we checking (destination vs origin)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')

get_stats_most_prevalent(
    get_agg_matching_scores_Skills(skill_similarities_all, n_trans),
    10)


,id,preferred_label,code,counts,prevalence,similarity,stdev
37,2052,greet guests,S3.4.4,4,0.211,0.000,0.000000
169,11012,have computer literacy,S5.5.0,4,0.211,0.000,0.000000
17,832,comply with food safety and hygiene,S3.3.1,3,0.158,0.001,0.000000
122,8440,maximise sales revenues,S1.6.1,4,0.211,0.003,0.003464
131,8831,ensure compliance with purchasing and contract...,S3.3.3,4,0.211,0.148,0.060000
13,710,communicate with customers,S3.4.1,4,0.211,0.802,0.000000
33,1809,maintain relationship with customers,S1.2.3,8,0.421,0.998,0.000000
93,6288,maintain customer service,A1.12.3,8,0.421,0.998,0.000000
82,5558,identify customer's needs,S1.7.2,7,0.368,0.998,0.000000
50,3248,maintain relationship with suppliers,S1.2.3,5,0.263,0.998,0.000000


In [378]:
get_stats_most_prevalent(
    get_agg_matching_scores_Clusters(skill_similarities_all, 'level_2', n_trans),
    10)

,code,title,counts,prevalence,similarity,stdev
23,S4.1,developing objectives and strategies,6,1.000,0.000,0.000548
11,S1.4,presenting information,5,0.833,0.000,0.000548
25,S4.3,allocating and controlling resources,3,0.500,0.000,0.000000
24,S4.2,"organising, planning and scheduling work and a...",6,1.000,0.001,0.001211
27,S4.8,supervising people,4,0.667,0.006,0.008505
20,S2.8,"monitoring, inspecting and testing",6,1.000,0.236,0.409215
9,S1.2,liaising and networking,7,1.167,0.288,0.335209
21,S3.3,protecting and enforcing,8,1.333,0.374,0.516514
22,S3.4,providing information and support to the publi...,6,1.000,0.418,0.458740
6,S1.1,negotiating,6,1.000,0.499,0.546627


### 3.0.2 Hotel concierge to sales & services clusters

In [399]:
# Lists of "transition skills" to merge into a dataframe
trans_to_merge = trans_clust[trans_clust.origin_id==job_i]
trans_to_merge = trans_to_merge[trans_to_merge.destination_level_1==2]
lists_to_merge = trans_to_merge.index

# Number of transitions we have
n_trans = len(lists_to_merge)

# SET UP which skills are we checking (destination vs origin)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')

df_concierge_to_services = get_stats_most_prevalent(
    get_agg_matching_scores_Skills(skill_similarities_all, n_trans),
    10)


In [400]:
df_concierge_to_services

,id,preferred_label,code,counts,prevalence,similarity,stdev
16,1127,characteristics of products,K7.2.2,2,0.2,0.000,0.000000
8,710,communicate with customers,S3.4.1,4,0.4,0.240,0.313341
44,5206,assist customers,S3.4.4,2,0.2,0.265,0.373352
31,3248,maintain relationship with suppliers,S1.2.3,2,0.2,0.444,0.618011
0,31,guarantee customer satisfaction,A1.12.3,3,0.3,0.707,0.186484
57,6288,maintain customer service,A1.12.3,5,0.5,0.998,0.000000
25,2052,greet guests,S3.4.4,4,0.4,0.998,0.000000
27,2187,handle customer complaints,S1.1.3,3,0.3,0.998,0.000000
12,832,comply with food safety and hygiene,S3.3.1,3,0.3,0.998,0.000000
84,9640,assist clients with special needs,S3.4.4,2,0.2,0.998,0.000000


In [398]:
get_stats_most_prevalent(
    get_agg_matching_scores_Clusters(skill_similarities_all, 'level_2', n_trans),
    10)

,code,title,counts,prevalence,similarity,stdev
37,S5.5,accessing and analysing digital data,5,0.5,0.000,0.000000
29,S4.1,developing objectives and strategies,4,0.4,0.000,0.000000
25,S2.8,"monitoring, inspecting and testing",4,0.4,0.000,0.000000
7,K4.1,business and administration,7,0.7,0.027,0.069928
30,S4.2,"organising, planning and scheduling work and a...",8,0.8,0.090,0.170895
14,S1.2,liaising and networking,13,1.3,0.224,0.419903
27,S3.3,protecting and enforcing,9,0.9,0.444,0.525676
28,S3.4,providing information and support to the publi...,19,1.9,0.448,0.465291
13,S1.1,negotiating,5,0.5,0.599,0.546627
0,A1.12,manage quality,12,1.2,0.673,0.420666


## 3.1 COVID-19 unsafe to safer occupations

In [296]:
data.clusters_level_2[data.clusters_level_2.level_1==2]

,level_1,level_2,keywords,keywords_manual,comments,comments_risk,counts
11,2,11,"attendant, cleaner, chef, restaurant, head, ro...","food, cleaning & customer services workers","includes food, cleaning, hospitality & beauty","could be broken down into hospitality, cleanin...",97
12,2,12,"representative, sale, sale representative, ser...",customer representatives,-,-,62
13,2,13,"shop manager, shop, manager, equipment shop, e...",shop & services managers,-,-,61
14,2,14,"specialised seller, seller, specialised, equip...",retail & sales workers,"includes personal shoppers, stylists",-,54
15,2,15,"merchant, wholesale merchant, wholesale, produ...",wholesale merchants,-,-,48


In [363]:
# len(trans_clust)

In [306]:
# Calculate the median high risk occupation's covid risk
high_risk_occ = data.occ_top[data.occ_top.risk_cat_label=='Very high risk']
med_HR_covid_score = high_risk_occ.covid_score.median()

# select_occ = high_risk_occ[(high_risk_occ.level_1==2) & (high_risk_occ.covid_score > med_HR_covid_score)].id.to_list()


### 3.1.1 To sales & services

In [374]:
n_trans_to_safe_covid =  len(trans_covid[
    (trans_covid.origin_level_1==2)
    & (trans_covid.destination_level_1==2)
    & ((trans_covid.destination_covid<med_HR_covid_score) | (trans_covid.destination_covid<trans_covid.origin_covid))])

n_trans = len(trans_covid[
    (trans_covid.origin_level_1==2)
    & (trans_covid.destination_level_1==2)])

print(n_trans, n_trans_to_safe_covid)

348 133


In [368]:
trans_to_merge = trans_covid[
    (trans_covid.origin_level_1==2)
    & (trans_covid.destination_level_1==2)]

In [375]:
trans_to_merge = trans_covid[trans_covid.origin_level_1==2]
trans_to_merge = trans_to_merge[(trans_to_merge.destination_level_1==2) #&
#                                 & (trans_to_merge.destination_covid<med_HR_covid_score)
                               ]
lists_to_merge = trans_to_merge.index

# Number of transitions we have
n_trans = len(lists_to_merge)

# SET UP which skills are we checking (destination vs origin)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')

get_stats_most_prevalent(
    get_agg_matching_scores_Skills(skill_similarities_all, n_trans),
    15)

,id,preferred_label,code,counts,prevalence,similarity,stdev
335,8440,maximise sales revenues,S1.6.1,65,0.187,0.133,0.327407
77,1776,manage budgets,S4.3.1,62,0.178,0.214,0.407695
118,2778,manage staff,S4.8.1,85,0.244,0.266,0.347582
96,2187,handle customer complaints,S1.1.3,75,0.216,0.361,0.481162
460,11012,have computer literacy,S5.5.0,79,0.227,0.417,0.495332
471,11313,product comprehension,K4.1.6,72,0.207,0.438,0.483611
47,1127,characteristics of products,K7.2.2,78,0.224,0.450,0.479946
560,13210,characteristics of services,K4.1.6,78,0.224,0.463,0.473276
402,9777,implement sales strategies,S1.6.2,62,0.178,0.590,0.445233
125,3248,maintain relationship with suppliers,S1.2.3,64,0.184,0.667,0.374838


In [361]:
get_stats_most_prevalent(
    get_agg_matching_scores_Clusters(skill_similarities_all, 'level_2', n_trans),
    10)

,code,title,counts,prevalence,similarity,stdev
54,S4.2,"organising, planning and scheduling work and a...",224,0.644,0.104,0.253292
46,S2.8,"monitoring, inspecting and testing",189,0.543,0.124,0.286628
48,S3.3,protecting and enforcing,392,1.126,0.172,0.330986
53,S4.1,developing objectives and strategies,234,0.672,0.212,0.349208
27,S1.1,negotiating,262,0.753,0.242,0.385030
18,K4.1,business and administration,467,1.342,0.277,0.407379
37,S1.6,"promoting, selling and purchasing",456,1.310,0.337,0.413643
49,S3.4,providing information and support to the publi...,447,1.284,0.369,0.442671
33,S1.2,liaising and networking,383,1.101,0.512,0.439809
1,A1.12,manage quality,299,0.859,0.616,0.427256


### 3.1.2 To business admin

In [362]:
trans_to_merge = trans_covid[trans_covid.origin_level_1==2]
trans_to_merge = trans_to_merge[(trans_to_merge.destination_level_1==1) 
#                                & (trans_to_merge.destination_covid<med_HR_covid_score)
                               ]
lists_to_merge = trans_to_merge.index

# Number of transitions we have
n_trans = len(lists_to_merge)

# SET UP which skills are we checking (destination vs origin)
skill_similarities_all = merge_lists(lists_to_merge, skills_type='destination')

get_stats_most_prevalent(
    get_agg_matching_scores_Skills(skill_similarities_all, n_trans),
    15)

,id,preferred_label,code,counts,prevalence,similarity,stdev
262,5472,corporate social responsibility,K4.1.3,26,0.193,0.000,0.000000
470,9852,plan health and safety procedures,S4.1.5,19,0.141,0.000,0.000918
577,12026,train employees,S1.3.3,16,0.119,0.002,0.002733
596,12672,liaise with managers,S1.2.1,21,0.156,0.034,0.043252
604,12932,follow company standards,S3.3.3,26,0.193,0.046,0.064947
510,10770,plan medium to long term objectives,S4.1.0,20,0.148,0.250,0.443373
480,10065,develop professional network,S1.2.3,28,0.207,0.256,0.285208
521,10988,present reports,S1.4.1,22,0.163,0.262,0.384232
6,160,create solutions to problems,S1.9.1,28,0.207,0.345,0.459236
379,7794,manage contracts,S1.1.1,19,0.141,0.409,0.464426


### X

In [612]:
x = [495, 3676, 6443]
x =  [7523, 12655]
data.skills.loc[x][['preferred_label']]

,preferred_label
7523,maintain ship logs
12655,maintain voyage logs


In [610]:
for y in data.skills.loc[x].description.to_list():
    print(y)
    print()

Maintain written records of events and activities on a ship

Maintain a written records of events during a ship or aeroplane voyage.



In [614]:
data.skills.loc[8143].description

'Design, modify and maintain systems that provide an organised collection of job descriptions.'

In [613]:
data.skills[data.skills.preferred_label.str.contains('occupational')]

,concept_type,concept_uri,skill_type,reuse_level,preferred_label,alt_labels,description,id,code,skill_category,level_1,level_2,level_3,level_title,level_href,level_uri
806,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0f69d0b1-b2a4...,knowledge,occupation-specific,occupational medicine,occupational and enivronmental medicine\noccup...,Occupational medicine is a medical specialty m...,806,K9.1.5,K,K9,K9.1,K9.1.5,therapy and rehabilitation,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/isced-f/0915
1469,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/1bd24c22-f065...,knowledge,cross-sector,occupational physiology,ergonomics associated with work\nphysiology of...,The complex physiology of specific jobs and it...,1469,K9.1.5,K,K9,K9.1,K9.1.5,therapy and rehabilitation,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/isced-f/0915
2413,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/2cbb31ac-2403...,skill/competence,cross-sector,investigate occupational injuries,investigate occupation-related illnesses\nasse...,"Assess, manage, and report cases of occupation...",2413,S2.1.1,S,S2,S2.1,S2.1.1,conducting investigations,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S2.1.1
4304,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/50ed4d18-60a8...,knowledge,cross-sector,occupational therapy theories,theoretical foundations of occupational therap...,The fundamental theories that underlie occupat...,4304,K9.1.5,K,K9,K9.1,K9.1.5,therapy and rehabilitation,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/isced-f/0915
4806,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/5a70f4b8-0e70...,skill/competence,occupation-specific,minimise occupational hazards in dentistry pra...,attempt to eliminate occupational hazards in d...,Understand and implement procedures to minimis...,4806,S3.3.1,S,S3,S3.3,S3.3.1,complying with health and safety procedures,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S3.3.1
7042,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/8460496d-aa54...,skill/competence,sector-specific,apply techniques of occupational therapy,perform occupational therapy\napply occupation...,"Apply occupational therapy techniques, such as...",7042,S3.2.4,S,S3,S3.2,S3.2.4,providing psychological and occupational thera...,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S3.2.4
8143,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/98f14bc9-4eb6...,skill/competence,occupation-specific,develop occupational classification systems,design systems for occupational classification...,"Design, modify and maintain systems that provi...",8143,S2.3.0,S,S2,S2.3,S2.3.0,managing information,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S2.3.0
9370,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/b157439a-3b5f...,knowledge,sector-specific,occupational science,study of everyday activity\noccupational scien...,The study of everyday activity including the b...,9370,K3.1.4,K,K3,K3.1,K3.1.4,sociology and cultural studies,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/isced-f/0314
9955,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/bd7540e1-ae1f...,skill/competence,occupation-specific,manage occupational therapy students,supervise occupational therapy students\nmanag...,Supervise and manage the work of occupational ...,9955,S4.8.1,S,S4,S4.8,S4.8.1,supervising a team or group,https://ec.europa.eu/esco/api/resource/concept...,http://data.europa.eu/esco/skill/S4.8.1
12546,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ef1b72a4-7f39...,skill/competence,cross-sector,educate employees on occupational hazards,educate staff on occupational hazards\nadvise ...,Provide information and advice to employees re...,12546,S1.3.2,S,S1,S1.3,S